# Chapter 8: Feature Diagnostics
AFML focus: feature attribution and regime-aware diagnostics.


In [1]:
import math
import matplotlib.pyplot as plt
import polars as pl
import openquant
import sys
from pathlib import Path
sys.path.insert(0, str(Path('notebooks/python/scripts').resolve()))
from afml_chapter_utils import (
    fetch_panel,
    simple_returns,
    probs_and_sides_from_momentum,
    timestamps_from_dates,
    lag_corr,
    fracdiff_ffd,
)

panel = fetch_panel(window=900)
dates = panel['date'].to_list()
uso = panel['USO'].to_list()
uso_ret = simple_returns(uso)
probs, sides = probs_and_sides_from_momentum(uso)
timestamps = timestamps_from_dates(dates)
asset_names = ['USO', 'BNO', 'XLE', 'GLD', 'UNG']
asset_prices = panel.select(asset_names).rows()
print('rows', panel.height, 'range', dates[0], dates[-1])


rows 900 range 2022-07-08 2026-02-06


In [2]:
features = ['oil_mom_5d', 'xle_lead_1d', 'gld_hedge', 'term_proxy', 'vol_proxy']
importance = [0.34, 0.24, 0.14, 0.17, 0.11]
std = [0.03, 0.02, 0.015, 0.018, 0.012]
fi = openquant.viz.prepare_feature_importance_payload(features, importance, std=std)
reg = openquant.viz.prepare_regime_payload(timestamps[-250:], uso_ret[-250:], threshold=0.0)

plt.figure(figsize=(8,4))
plt.bar(fi['x'], fi['y'])
plt.title('Chapter 8: Feature Importance Payload')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,3))
plt.plot(reg['score'], lw=1)
plt.plot(reg['regime'], lw=1)
plt.title('Chapter 8: Regime Score vs Regime Label')
plt.tight_layout()
plt.show()

print('top feature', fi['x'][0])


top feature oil_mom_5d


## Interpretation
Feature payloads and regime overlays provide a practical diagnostics layer for model audits and instability tracking.

